In [1]:
import os
os.getcwd()

'/content'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
dir = os.path.join('drive','My Drive','Colab Notebooks','STAT628_module3')
dir

'drive/My Drive/Colab Notebooks/STAT628_module3'

### Load data

In [0]:
import torch
from torchtext import data
from torchtext import datasets
from torchtext.data import TabularDataset

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [0]:
txt_datafields = [("text", TEXT), # we won't be needing the id, so we pass in None as the field
                  ("stars", LABEL)]
txt = TabularDataset(
           path=os.path.join(dir,"dataset","RNN_total.csv"), # the file path
           format='csv',
           skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
           fields=txt_datafields)

In [6]:
print(f'Number of testing examples: {len(txt)}')

Number of testing examples: 123397


In [0]:
import random

train_data, other = txt.split(random_state = random.seed(SEED))

In [0]:
valid_data,test_data = other.split(split_ratio = 0.5,random_state = random.seed(SEED))

In [9]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 86378
Number of validation examples: 18510
Number of testing examples: 18509


In [10]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_) # how to init out-of-vocab words see https://torchtext.readthedocs.io/en/latest/vocab.html?highlight=unk_init

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 398103/400000 [00:20<00:00, 19962.63it/s]

In [11]:
TEXT.vocab.vectors[:20]

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.0795,  0.3017,  0.0795,  ..., -0.1673,  0.6534,  0.3660],
        [-0.4989,  0.7660,  0.8975,  ..., -0.4118,  0.4054,  0.7850],
        [-0.5706,  0.4418,  0.7010,  ..., -0.6610,  0.4720,  0.3725]])

In [12]:
TEXT.vocab.itos[:20]

['<unk>',
 '<pad>',
 '.',
 'the',
 'and',
 ',',
 'i',
 'a',
 'to',
 'it',
 '!',
 'was',
 'of',
 'is',
 'in',
 ' ',
 'for',
 'they',
 'you',
 'this']

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text),  ####  add
    sort_within_batch = False,
    device = device)

In [14]:
for i in test_iterator:
  print(i.stars.long())
  #print(i.text)
  break

tensor([0, 0, 2, 0, 0, 2, 4, 1, 2, 1, 3, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 3, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
        2, 4, 4, 1, 0, 0, 1, 2, 2, 0, 1, 1, 0, 0, 1, 0], device='cuda:0')


### build the model

In [15]:
torch.cuda.is_available()

True

In [0]:
import torch.nn as nn

In [0]:

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, enforce_sorted=False)  ##  edited
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  ## edited

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [19]:
len(TEXT.vocab)

25002

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,812,909 trainable parameters


In [21]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [22]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.1850, -0.6996, -0.2782,  ..., -0.4959, -0.1750, -0.3778],
        [-0.4955, -0.0474,  0.4095,  ...,  1.1165, -0.1838, -0.8457],
        [-0.6854,  0.4087, -0.5241,  ..., -0.3518, -0.7017,  0.4724]])

In [23]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.1850, -0.6996, -0.2782,  ..., -0.4959, -0.1750, -0.3778],
        [-0.4955, -0.0474,  0.4095,  ...,  1.1165, -0.1838, -0.8457],
        [-0.6854,  0.4087, -0.5241,  ..., -0.3518, -0.7017,  0.4724]])


In [24]:
PAD_IDX

1

### Train the model

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.stars.long())
        
        acc = categorical_accuracy(predictions, batch.stars.long())
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.stars.long())
            
            acc = categorical_accuracy(predictions, batch.stars.long())

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [31]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 3m 45s
	Train Loss: 1.046 | Train Acc: 55.10%
	 Val. Loss: 0.883 |  Val. Acc: 61.82%
Epoch: 02 | Epoch Time: 3m 46s
	Train Loss: 0.831 | Train Acc: 63.87%
	 Val. Loss: 0.774 |  Val. Acc: 66.54%
Epoch: 03 | Epoch Time: 3m 47s
	Train Loss: 0.770 | Train Acc: 66.62%
	 Val. Loss: 0.747 |  Val. Acc: 67.07%
Epoch: 04 | Epoch Time: 3m 48s
	Train Loss: 0.731 | Train Acc: 68.46%
	 Val. Loss: 0.745 |  Val. Acc: 66.75%
Epoch: 05 | Epoch Time: 3m 48s
	Train Loss: 0.698 | Train Acc: 69.96%
	 Val. Loss: 0.726 |  Val. Acc: 68.46%
Epoch: 06 | Epoch Time: 3m 51s
	Train Loss: 0.671 | Train Acc: 71.15%
	 Val. Loss: 0.732 |  Val. Acc: 68.45%
Epoch: 07 | Epoch Time: 3m 50s
	Train Loss: 0.647 | Train Acc: 72.58%
	 Val. Loss: 0.738 |  Val. Acc: 68.80%
Epoch: 08 | Epoch Time: 3m 50s
	Train Loss: 0.626 | Train Acc: 73.32%
	 Val. Loss: 0.726 |  Val. Acc: 69.00%
Epoch: 09 | Epoch Time: 3m 54s
	Train Loss: 0.601 | Train Acc: 74.58%
	 Val. Loss: 0.755 |  Val. Acc: 68.54%
Epoch: 10 | Epoch T

### test accuracy

In [32]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.721 | Test Acc: 68.65%


In [0]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    pre_class = prediction.argmax()
    return LABEL.vocab.itos[pre_class]

In [34]:
predict_sentiment(model, "I like the flavor")

'5.0'

In [35]:
predict_sentiment(model, "a little expensive")

'3.0'

In [36]:
predict_sentiment(model, "The service sucks")

'1.0'

## Prediction

In [0]:
import numpy as np
import pandas as pd

In [47]:
food = pd.read_csv(os.path.join(dir,'dataset','food_sentence.csv'))
food.head(2)

,business_id,text
0,irft4YkdNsww4DNf_Aftew,pretty good froyo place with all the toppings ...
1,irft4YkdNsww4DNf_Aftew,very tasty even with the added protein it does...


In [58]:
food_pre = list(map(lambda x: predict_sentiment(model, x),food.text))
food_stars = pd.concat([food,pd.DataFrame({'stars':food_pre})],axis = 1)
food_stars.head()

,business_id,text,stars
0,irft4YkdNsww4DNf_Aftew,pretty good froyo place with all the toppings ...,4.0
1,irft4YkdNsww4DNf_Aftew,very tasty even with the added protein it does...,5.0
2,GWO87Y-IqL54_Ijx6hTYAQ,they have tons of cool options and taste bud b...,5.0
3,GWO87Y-IqL54_Ijx6hTYAQ,one of my favorites is the coconut chip.,5.0
4,GWO87Y-IqL54_Ijx6hTYAQ,not that i'm saying bruster's is bad or anythi...,2.0
